<a href="https://colab.research.google.com/github/orilevi2809/DL/blob/main/project/all_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from PIL import Image
import pickle
import numpy as np

!pip install timm
from torch.utils.data import Dataset
from google.colab import drive
import timm
import torch.nn as nn
from skimage.metrics import structural_similarity as ssim
from torch.utils.data import Dataset
from torch.optim import lr_scheduler

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
Mounted at /content/drive


In [10]:
IMG_SIZE = 224 #TODO: check if we can minize the size of the image e.g. 32*32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img_dir = "./drive/MyDrive/labels/type_labels_base/type_labels_base"
labels_dict = {"crimp": 0, "pinch": 1, "pocket": 2, "sloper": 3, "jug": 4}#,"jib":5}#, "edge": 5 , : 2
inv_labels_dict = {0 : "crimp",  1 : "pinch",  2 : "pocket",  3: "sloper",   4 : "jug"}#,5:"jib"}#, "edge": 5 ,
class CustomLayer(nn.Module):
    def __init__(self, dropout_rate, in_features, out_features):
        super().__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(in_features, out_features) # Change this
        self.softmax = nn.Softmax(dim=1)
        self.out_features = out_features

    def forward(self, x):
        x = self.dropout(x)
        x = self.linear(x) # Change this
        #x = self.softmax(x)
        return x
class ImagePathDataset(Dataset):
    def __init__(self, images, labels, paths):
        self.images = images
        self.labels = labels
        self.paths = paths

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx], self.paths[idx]

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomRotation(degrees=(0, 30)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
noise_factor = 0.3  # Define the intensity of the noise

train_transform2 = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=.3, hue=.3),
    transforms.RandomRotation(degrees=(-40, 40)),
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor: tensor + torch.randn_like(tensor) * noise_factor),  # Modified lambda function to add noise to a Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_transform4 = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ColorJitter(brightness=.3, hue=.4),
    transforms.RandomAutocontrast(),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomRotation(degrees=(-50, 50)),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform5 = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ColorJitter(brightness=.3, hue=.3),
    transforms.RandomRotation(degrees=(-25, 25)),
    transforms.ToTensor(),
    # Modified lambda function to add noise to a Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
def save_model(model, model_file):
    # Save the model's state dictionary to disk
    torch.save(model.state_dict(), model_file)


In [4]:
def load_images_paths(img_dir, labels):
    data = []
    target = []
    label_count = {}  # Dictionary to store the count of images for each label
    for label in labels:
        label_dir = os.path.join(img_dir, str(label))
        count = 0  # Initialize the count for the current label
        for filename in os.listdir(label_dir):
            if filename.endswith(".jpg") or filename.endswith(".jpeg")or filename.endswith(".png"):
                img_path = os.path.join(label_dir, filename)
                data.append(img_path)
                target.append(labels_dict[label])
                count += 1  # Increment the count for the current label
        label_count[label] = count  # Store the count in the dictionary

    # Print the count of images for each label
    for label, count in label_count.items():
        print(f"Number of images for label '{label}': {count}")
    return data, target

In [5]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    if isinstance(img, torch.Tensor):
        img = img.numpy()
        img = np.transpose(img, (1, 2, 0))
        img = (img * 255).astype(np.uint8)
        plt.imshow(img)
    else:  # Assuming img is a PIL.Image object
        plt.imshow(img)

    plt.show()


In [6]:
def load_image_data(img_paths, labels, transform, augment_unbalanced=False):
    if augment_unbalanced:
        data = []
        target = []
        path_list = []
        label_counts = {label: 0 for label in set(labels)}
        for img_path, label in zip(img_paths, labels):
            img = Image.open(img_path).convert("RGB")
            img = test_transform(img)
            data.append(img)
            path_list.append(img)
            target.append(label)
            label_counts[label] += 1

        is_unbalanced = any(count < max(label_counts.values()) for count in label_counts.values())
        print(label_counts)

        if is_unbalanced:
            print("unbalanced")
            augmented_data = []
            augmented_target = []
            path_list = []
            #TODO: add while all equal
            max_count = max(label_counts.values())
            for img_path, label in zip(img_paths, labels):
                img = Image.open(img_path).convert("RGB")
                augmented_data.append(test_transform(img))
                augmented_target.append(label)
                path_list.append(img_path)
                for i in range(1):
                    if label_counts[label] < 1*3*max_count:
                        augmented_img = train_transform4(img)
                        augmented_data.append(augmented_img)
                        augmented_target.append(label)
                        path_list.append(img_path)
                        label_counts[label] += 1
                    if label_counts[label] < 1*3*max_count:
                        augmented_img = train_transform(img)
                        augmented_data.append(augmented_img)
                        augmented_target.append(label)
                        path_list.append(img_path)
                        label_counts[label] += 1
                    if label_counts[label] < 1*3*max_count:
                        augmented_img = train_transform2(img)
                        augmented_data.append(augmented_img)
                        augmented_target.append(label)
                        path_list.append(img_path)
                        label_counts[label] += 1
                    if label_counts[label] < 1*3*max_count:
                        augmented_img = train_transform5(img)
                        augmented_data.append(augmented_img)
                        augmented_target.append(label)
                        path_list.append(img_path)
                        label_counts[label] += 1
            print(label_counts)
            return augmented_data, augmented_target,path_list
        return data, target,path_list
    else:
        if isinstance(img_paths[0], torch.Tensor):  # Check if data is already in tensor format
            return img_paths, labels, img_paths
        else:
            data = []
            target = []
            path_list = []
            for img_path, label in zip(img_paths, labels):
                img = Image.open(img_path).convert("RGB")
                img = test_transform(img)
                data.append(img)
                target.append(label)
                path_list.append(img_path)
            return data, target, path_list


In [7]:

def train_classifier(train_dataloader, val_dataloader, model, criterion, optimizer, device,scheduler,epochs):
    for epoch in range(epochs):  # 10 is the best!!
        running_loss = 0.0
        total_correct = 0
        total_samples = 0

        # Switch model to training mode
        model.train()

        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            # Calculate the number of correct predictions in the current batch
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()


            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        print(f"Epoch: {epoch + 1}, Learning Rate: {scheduler.get_last_lr()}")
        scheduler.step()
        train_accuracy = 100 * total_correct / total_samples

        # Switch model to evaluation mode for validation
        model.eval()

        val_correct = 0
        val_samples = 0
        val_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(val_dataloader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_correct += (predicted == labels).sum().item()
                val_samples += labels.size(0)

        val_accuracy = 100 * val_correct / val_samples

        print(f'Epoch: {epoch + 1}, Training loss: {running_loss / len(train_dataloader)}, Training accuracy: {train_accuracy}%, Validation loss: {val_loss / len(val_dataloader)}, Validation accuracy: {val_accuracy}%')

    print('Finished Training')
    return model

In [22]:
def evaluate_classifier(test_dataloader, model, device,img_paths_test):
    inverse_normalize = transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
                                              std=[1 / 0.229, 1 / 0.224, 1 / 0.225])

    model.eval()  # Switch model to evaluation mode
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels, paths in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if not torch.all(predicted == labels):
                filtered_imgs = images[predicted != labels]
                incorrect_indices = (predicted != labels).nonzero().squeeze()
                if incorrect_indices.dim() == 0:
                    incorrect_indices = incorrect_indices.unsqueeze(
                        0)  # Convert the 0D tensor to a 1D tensor with one element
                mispredicted_paths = [paths[i] for i in incorrect_indices]
                for i in range(len(filtered_imgs)):
                    mispredicted_image = filtered_imgs[i]  # Select the first mispredicted image
                    mispredicted_path = mispredicted_paths[i]
                    label = labels[predicted != labels][i].item()
                    prediction = predicted[predicted != labels][i].item()
                    path = mispredicted_path.split('/')[-1]
                    #plot_image(mispredicted_image,path, label, prediction, inverse_normalize)

            # Move tensors to CPU before converting to numpy arrays
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}%')
    # Generate classification report and confusion matrix
    report = classification_report(all_labels, all_predictions, zero_division=1)
    matrix = confusion_matrix(all_labels, all_predictions)
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(matrix)


In [11]:
img_paths, labels = load_images_paths(img_dir, labels_dict.keys())
# Split the data into train, validation, and test sets (e.g., 70%, 15%, 15%)
img_paths_train, img_paths_test, labels_train, labels_test = train_test_split(img_paths, labels, test_size=0.3,
                                                                              stratify=labels, random_state=42)
img_paths_val, img_paths_test, labels_val, labels_test = train_test_split(img_paths_test, labels_test,
                                                                          test_size=0.5, stratify=labels_test,
                                                                          random_state=42)
data_train, labels_train, img_paths_train = load_image_data(img_paths_train, labels_train, train_transform,
                                                            augment_unbalanced=True)
data_val, labels_val, img_paths_val = load_image_data(img_paths_val, labels_val, test_transform,
                                                      augment_unbalanced=False)
data_test1, labels_test1, img_paths_test = load_image_data(img_paths_test, labels_test, test_transform,
                                                            augment_unbalanced=False)
# Convert to tensors
data_train = torch.stack(data_train)
labels_train = torch.tensor(labels_train)
data_val = torch.stack(data_val)
labels_val = torch.tensor(labels_val)
data_test = torch.stack(data_test1)
labels_test = torch.tensor(labels_test1)
# Replace TensorDataset with ImagePathDataset
train_dataset = ImagePathDataset(data_train, labels_train, img_paths_train)
val_dataset = ImagePathDataset(data_val, labels_val, img_paths_val)
test_dataset = ImagePathDataset(data_test, labels_test, img_paths_test)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)



Number of images for label 'crimp': 183
Number of images for label 'pinch': 207
Number of images for label 'pocket': 234
Number of images for label 'sloper': 247
Number of images for label 'jug': 209
{0: 128, 1: 145, 2: 164, 3: 173, 4: 146}
unbalanced
{0: 519, 1: 519, 2: 519, 3: 519, 4: 519}


In [13]:
# Save the data loaders
pickle_dir = "./drive/MyDrive/pickles/"

with open(pickle_dir+'train_dataloader.pkl', 'wb') as f:
    pickle.dump(train_dataloader, f)

with open(pickle_dir+'val_dataloader.pkl', 'wb') as f:
    pickle.dump(val_dataloader, f)

with open(pickle_dir+'test_dataloader.pkl', 'wb') as f:
    pickle.dump(test_dataloader, f)

In [ ]:
# Load the data loaders
pickle_dir = "./drive/MyDrive/pickles/"
with open(pickle_dir+'train_dataloader.pkl', 'rb') as f:
    train_dataloader = pickle.load(f)

with open(pickle_dir+'val_dataloader.pkl', 'rb') as f:
    val_dataloader = pickle.load(f)

with open(pickle_dir+'test_dataloader.pkl', 'rb') as f:
    test_dataloader = pickle.load(f)

In [14]:
def get_model_mobileNet():
    # Load pre-trained MobileNetV2 model and replace the classifier layer
    model = models.mobilenet_v2(pretrained=True)
    dropout_rate = 0.3  # Specify the dropout rate
    in_features = model.classifier[-1].in_features  # Replace with the correct number of input features
    out_features = len(labels_dict)  # Replace with the correct number of output features
    additional_fc_layer = CustomLayer(dropout_rate, in_features, out_features)

    model.classifier[-1] = additional_fc_layer

    model = model.to(device)
    return model



In [15]:

def get_model_efficientNet():
    # Load pre-trained EfficientNet model and replace the classifier layer
    model = timm.create_model('efficientnet_b2', pretrained=True)

    # Define the additional FC layer with dropout
    dropout_rate = 0.3  # Specify the dropout rate
    additional_fc_layer = nn.Sequential(
      nn.Dropout(dropout_rate),
      nn.Linear(model.classifier.in_features, len(labels_dict)),  # Replace len(labels_dict) with the number of output classes
      #nn.Softmax(dim=1)  # Add the softmax activation layer
    )

    model.classifier = additional_fc_layer

    model = model.to(device)
    return model

In [30]:
def get_model_vgg():
    # Load pre-trained VGG16 model
    model = models.vgg16(pretrained=True)

    # Replace the classifier layer
    dropout_rate = 0.3  # Specify the dropout rate
    in_features = 4096  # Get the number of input features
    out_features = len(labels_dict)  # Replace with the correct number of output features
    additional_fc_layer = CustomLayer(dropout_rate, in_features, out_features)

    model.classifier[6] = additional_fc_layer

    model = model.to(device)
    return model

In [17]:

def get_model_densenet():
    # Load pre-trained DenseNet model and replace the classifier layer
    model = models.densenet201(pretrained=True)


    dropout_rate = 0.3  # Specify the dropout rate
    additional_fc_layer = nn.Sequential(
      nn.Dropout(dropout_rate),
      nn.Linear(model.classifier.in_features, len(labels_dict)),  # Replace len(labels_dict) with the number of output classes
      #nn.Softmax(dim=1)  # Add the softmax activation layer
    )
    model.classifier = additional_fc_layer  # assuming labels_dict is defined elsewhere

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model

In [18]:

def get_model_resnet():
    # Load pre-trained ResNet model and replace the classifier layer
    model = models.resnet152(pretrained=True)

    dropout_rate = 0.5  # Specify the dropout rate
    additional_fc_layer = nn.Sequential(
        #nn.Linear(model.fc.in_features, 1000),
        nn.Dropout(dropout_rate),
        nn.Linear(model.fc.in_features, len(labels_dict)),
        #nn.Softmax(dim=1)  # Add the softmax activation layer
    )
    model.fc = additional_fc_layer


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model

In [39]:

def get_model_alexnet():
    # Load pre-trained AlexNet model and replace the classifier layer
    model = models.alexnet(pretrained=True)

    dropout_rate = 0.3  # Specify the dropout rate
    additional_fc_layer = nn.Sequential(
      nn.Dropout(dropout_rate),
      nn.Linear(model.classifier[6].in_features, len(labels_dict)),  # Replace len(labels_dict) with the number of output classes
      #nn.Softmax(dim=1)  # Add the softmax activation layer
    )
    model.classifier[6].in_features = nn.Linear(model.classifier[6].in_features, len(labels_dict))  # assuming labels_dict is defined elsewhere

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model

In [20]:
def train_model(model,epochs,lr):
  # Define loss criterion and optimizer
  optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
  criterion = nn.CrossEntropyLoss()
  scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
  model = train_classifier(train_dataloader, val_dataloader, model, criterion, optimizer, device, scheduler, epochs)

  if not os.path.exists('hold_classifier_MobileNet.pth'):
      # Save the MobileNetV2 model
      model_file = 'hold_classifier_MobileNet.pth'
      save_model(model, model_file)

      # Load the model for inference
      model.load_state_dict(torch.load('hold_classifier_MobileNet.pth'))
  return model



In [ ]:

if not os.path.exists('hold_classifier_MobileNet.pth'):
    # Save the MobileNetV2 model
    model_file = 'hold_classifier_MobileNet.pth'
    save_model(model, model_file)

    # Load the model for inference
    model.load_state_dict(torch.load('hold_classifier_MobileNet.pth'))




In [24]:
model = get_model_mobileNet()
trained_model = train_model(model,epochs=10,lr = 0.00005)
evaluate_classifier(test_dataloader, trained_model, device,img_paths_test)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Learning Rate: [5e-05]
Epoch: 1, Training loss: 1.1887089071824, Training accuracy: 51.252408477842%, Validation loss: 0.5689842630000341, Validation accuracy: 83.33333333333333%
Epoch: 2, Learning Rate: [5e-05]
Epoch: 2, Training loss: 0.6666993739513251, Training accuracy: 75.79961464354528%, Validation loss: 0.49889782745213734, Validation accuracy: 81.48148148148148%
Epoch: 3, Learning Rate: [5e-05]
Epoch: 3, Training loss: 0.4322828610929159, Training accuracy: 84.50867052023122%, Validation loss: 0.4508064856664056, Validation accuracy: 83.95061728395062%
Epoch: 4, Learning Rate: [5e-06]
Epoch: 4, Training loss: 0.32355558422322456, Training accuracy: 89.28709055876686%, Validation loss: 0.41374443418213297, Validation accuracy: 85.80246913580247%
Epoch: 5, Learning Rate: [5e-06]
Epoch: 5, Training loss: 0.2687045674140637, Training accuracy: 91.52215799614643%, Validation loss: 0.4022814126773959, Validation accuracy: 86.41975308641975%
Epoch: 6, Learning Rate: [5e-06]

In [ ]:
model = get_model_mobileNet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.0350793908382285, Training accuracy: 61.40642303433001%, Validation loss: 1.5941693484783173, Validation accuracy: 56.41025641025641%
Epoch: 2, Training loss: 0.5694051333542528, Training accuracy: 79.67884828349945%, Validation loss: 1.457981362938881, Validation accuracy: 61.53846153846154%
Epoch: 3, Training loss: 0.4081995548873112, Training accuracy: 85.27131782945736%, Validation loss: 1.5894124507904053, Validation accuracy: 57.43589743589744%
Epoch: 4, Training loss: 0.2656496886549325, Training accuracy: 90.58693244739756%, Validation loss: 2.1410719007253647, Validation accuracy: 63.58974358974359%
Epoch: 5, Training loss: 0.221304417941077, Training accuracy: 92.19269102990033%, Validation loss: 1.494978904724121, Validation accuracy: 60.0%
Epoch: 6, Training loss: 0.22078728547384, Training accuracy: 92.41417497231451%, Validation loss: 2.253840982913971, Validation accuracy: 64.61538461538461%
Epoch: 7, Training loss: 0.1555251512548019, Training

In [25]:
model = get_model_efficientNet()
trained_model = train_model(model,epochs=10,lr=0.0001)
evaluate_classifier(test_dataloader, trained_model, device,img_paths_test)

Epoch: 1, Learning Rate: [0.0001]
Epoch: 1, Training loss: 1.0132694054566898, Training accuracy: 61.27167630057804%, Validation loss: 0.6326338797807693, Validation accuracy: 77.77777777777777%
Epoch: 2, Learning Rate: [0.0001]
Epoch: 2, Training loss: 0.4491519378354916, Training accuracy: 84.58574181117534%, Validation loss: 0.49617333497319904, Validation accuracy: 81.48148148148148%
Epoch: 3, Learning Rate: [0.0001]
Epoch: 3, Training loss: 0.25933087699401836, Training accuracy: 91.79190751445087%, Validation loss: 0.585948684031055, Validation accuracy: 80.8641975308642%
Epoch: 4, Learning Rate: [1e-05]
Epoch: 4, Training loss: 0.1567754502967, Training accuracy: 95.45279383429673%, Validation loss: 0.4869310795551255, Validation accuracy: 85.18518518518519%
Epoch: 5, Learning Rate: [1e-05]
Epoch: 5, Training loss: 0.12128244700340124, Training accuracy: 96.878612716763%, Validation loss: 0.4871495550587064, Validation accuracy: 83.95061728395062%
Epoch: 6, Learning Rate: [1e-05

In [ ]:
model = get_model_efficientNet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

Epoch: 1, Training loss: 0.872700945056718, Training accuracy: 68.6046511627907%, Validation loss: 0.9906402677297592, Validation accuracy: 73.84615384615384%
Epoch: 2, Training loss: 0.28199248298488816, Training accuracy: 91.5282392026578%, Validation loss: 1.5252069532871246, Validation accuracy: 67.17948717948718%
Epoch: 3, Training loss: 0.2294543370090682, Training accuracy: 92.24806201550388%, Validation loss: 1.9571730196475983, Validation accuracy: 65.12820512820512%
Epoch: 4, Training loss: 0.16999169683148121, Training accuracy: 94.73975636766335%, Validation loss: 1.313079059123993, Validation accuracy: 64.61538461538461%
Epoch: 5, Training loss: 0.1319006549130226, Training accuracy: 96.29014396456257%, Validation loss: 1.8295849561691284, Validation accuracy: 73.84615384615384%
Epoch: 6, Training loss: 0.0981348352185611, Training accuracy: 97.84053156146179%, Validation loss: 1.3715955913066864, Validation accuracy: 76.41025641025641%
Epoch: 7, Training loss: 0.098916076

In [31]:
model = get_model_VGG()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: ignored

In [33]:
model = get_model_resnet()
trained_model = train_model(model,epochs=10,lr=0.00005)
evaluate_classifier(test_dataloader, trained_model, device,img_paths_test)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Learning Rate: [5e-05]
Epoch: 1, Training loss: 0.8745788624194952, Training accuracy: 66.31984585741812%, Validation loss: 0.7643266614703905, Validation accuracy: 74.07407407407408%
Epoch: 2, Learning Rate: [5e-05]
Epoch: 2, Training loss: 0.42185184322870695, Training accuracy: 85.20231213872832%, Validation loss: 0.483121027238667, Validation accuracy: 79.62962962962963%
Epoch: 3, Learning Rate: [5e-05]
Epoch: 3, Training loss: 0.25279112823594074, Training accuracy: 92.06165703275529%, Validation loss: 0.4389472111527409, Validation accuracy: 82.09876543209876%
Epoch: 4, Learning Rate: [5e-06]
Epoch: 4, Training loss: 0.15925277252609912, Training accuracy: 94.99036608863199%, Validation loss: 0.36632786757711855, Validation accuracy: 87.03703703703704%
Epoch: 5, Learning Rate: [5e-06]
Epoch: 5, Training loss: 0.08685519260712542, Training accuracy: 97.91907514450867%, Validation loss: 0.3471059493188347, Validation accuracy: 87.03703703703704%
Epoch: 6, Learning Rate: [

In [ ]:
model = get_model_resnet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.1851976352982816, Training accuracy: 57.308970099667775%, Validation loss: 1.3689427421643183, Validation accuracy: 57.94871794871795%
Epoch: 2, Training loss: 0.680096819073753, Training accuracy: 75.91362126245848%, Validation loss: 0.8659154222561762, Validation accuracy: 70.76923076923077%
Epoch: 3, Training loss: 0.4324862684032558, Training accuracy: 84.9390919158361%, Validation loss: 1.261734196772942, Validation accuracy: 63.58974358974359%
Epoch: 4, Training loss: 0.28072028928559967, Training accuracy: 90.2547065337763%, Validation loss: 1.0815186248375819, Validation accuracy: 68.71794871794872%
Epoch: 5, Training loss: 0.2183475135372276, Training accuracy: 92.52491694352159%, Validation loss: 1.4864197786037738, Validation accuracy: 66.66666666666667%
Epoch: 6, Training loss: 0.20954040523651427, Training accuracy: 93.02325581395348%, Validation loss: 1.176123123902541, Validation accuracy: 67.17948717948718%
Epoch: 7, Training loss: 0.141680413

In [40]:
model = get_model_alexnet()
trained_model = train_model(model,epochs=10,lr=0.00005)
evaluate_classifier(test_dataloader, trained_model, device,img_paths_test)

Epoch: 1, Learning Rate: [5e-05]
Epoch: 1, Training loss: 1.6084787657627693, Training accuracy: 46.859344894026975%, Validation loss: 0.8037562725089845, Validation accuracy: 70.37037037037037%
Epoch: 2, Learning Rate: [5e-05]
Epoch: 2, Training loss: 0.6990604210816896, Training accuracy: 72.52408477842003%, Validation loss: 0.7570531417926153, Validation accuracy: 79.01234567901234%
Epoch: 3, Learning Rate: [5e-05]
Epoch: 3, Training loss: 0.4505840495458016, Training accuracy: 82.92870905587668%, Validation loss: 0.9089510185377938, Validation accuracy: 72.8395061728395%
Epoch: 4, Learning Rate: [5e-06]
Epoch: 4, Training loss: 0.2356823197207772, Training accuracy: 92.06165703275529%, Validation loss: 0.7892801215251287, Validation accuracy: 76.54320987654322%
Epoch: 5, Learning Rate: [5e-06]
Epoch: 5, Training loss: 0.17901525038867616, Training accuracy: 94.29672447013488%, Validation loss: 0.8141397244873501, Validation accuracy: 75.30864197530865%
Epoch: 6, Learning Rate: [5e-

In [ ]:
model = get_model_alexnet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.2179580529530842, Training accuracy: 53.98671096345515%, Validation loss: 1.0619439312389918, Validation accuracy: 65.64102564102564%
Epoch: 2, Training loss: 0.6933513932060777, Training accuracy: 74.08637873754152%, Validation loss: 1.0820266689573015, Validation accuracy: 68.2051282051282%
Epoch: 3, Training loss: 0.46243443536131007, Training accuracy: 82.89036544850498%, Validation loss: 1.0185822333608354, Validation accuracy: 67.17948717948718%
Epoch: 4, Training loss: 0.31267687077062173, Training accuracy: 88.70431893687707%, Validation loss: 1.2470426048551286, Validation accuracy: 63.58974358974359%
Epoch: 5, Training loss: 0.19245363902627377, Training accuracy: 93.63233665559247%, Validation loss: 1.388728107724871, Validation accuracy: 62.56410256410256%
Epoch: 6, Training loss: 0.15043253039843157, Training accuracy: 95.01661129568106%, Validation loss: 1.6416954398155212, Validation accuracy: 63.07692307692308%
Epoch: 7, Training loss: 0.10527

In [43]:
model = get_model_densenet()
trained_model = train_model(model,epochs=10,lr = 0.00003)
evaluate_classifier(test_dataloader, trained_model, device,img_paths_test)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Learning Rate: [1e-05]
Epoch: 1, Training loss: 1.4515467194410472, Training accuracy: 40.23121387283237%, Validation loss: 1.071089086078462, Validation accuracy: 64.19753086419753%
Epoch: 2, Learning Rate: [1e-05]
Epoch: 2, Training loss: 0.9923415849759029, Training accuracy: 66.6281310211946%, Validation loss: 0.7179636827536992, Validation accuracy: 78.39506172839506%
Epoch: 3, Learning Rate: [1e-05]
Epoch: 3, Training loss: 0.6994571438202491, Training accuracy: 77.57225433526011%, Validation loss: 0.5449585701738086, Validation accuracy: 81.48148148148148%
Epoch: 4, Learning Rate: [1.0000000000000002e-06]
Epoch: 4, Training loss: 0.5517344456911087, Training accuracy: 84.08477842003853%, Validation loss: 0.5249591604584739, Validation accuracy: 82.71604938271605%
Epoch: 5, Learning Rate: [1.0000000000000002e-06]
Epoch: 5, Training loss: 0.5246119435475423, Training accuracy: 84.47013487475915%, Validation loss: 0.5176957157396135, Validation accuracy: 83.33333333333333

In [ ]:
model = get_model_densenet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 0.8734383624175499, Training accuracy: 66.38981173864894%, Validation loss: 1.2653553783893585, Validation accuracy: 68.2051282051282%
Epoch: 2, Training loss: 0.2656413010995963, Training accuracy: 92.24806201550388%, Validation loss: 1.11929751932621, Validation accuracy: 72.3076923076923%
Epoch: 3, Training loss: 0.10597261715808819, Training accuracy: 97.28682170542636%, Validation loss: 1.0208176374435425, Validation accuracy: 74.35897435897436%
Epoch: 4, Training loss: 0.046826563004789684, Training accuracy: 99.16943521594685%, Validation loss: 1.26620202511549, Validation accuracy: 74.35897435897436%
Epoch: 5, Training loss: 0.03552374853913126, Training accuracy: 99.16943521594685%, Validation loss: 1.0763324946165085, Validation accuracy: 71.7948717948718%
Epoch: 6, Training loss: 0.01930790259663401, Training accuracy: 99.6124031007752%, Validation loss: 1.3661493062973022, Validation accuracy: 73.84615384615384%
Epoch: 7, Training loss: 0.0115702782

In [ ]:
labels_dict = {"crimp": 0, "pinch": 1, "pocket": 2, "jug": 3, "edge": 4, "sloper": 5}
